# Cleaning and Merging the Data

Open Streets Data:  

- Downloaded from 311 Open Data portal (Vector file of lines for each open street)
- Calculated mean coordinates for each line (i.e. each registered Open Street) in QGIS

Census data: 

- From ACS survey
- Table B19013 from ACS 5yr (2021) downloaded from Census Reporter

Census Tract data for NYC 

- Downloaded [here](https://www.nyc.gov/site/planning/data-maps/open-data/census-download-metadata.page) via NYC portal
- Originally as a shapefile then converted to csv in QGIS


In [1]:
import pandas as pd

In [2]:
! ls

README.md                             mean_coords_table_with_geos.csv
acs2021_5yr_B19013_14000US36047030600 ny_census_tracts.csv
analysis.ipynb                        ny_census_tracts.qmd
clean_merge.ipynb                     open_streets_count.csv
edited_complaints.csv                 predicted_open_streets
final_data.csv                        prediction.csv
geocode.ipynb                         requirements.txt
mean_coords_table.tsv


Reading the ACS income data

In [3]:
income = pd.read_csv('./acs2021_5yr_B19013_14000US36047030600/acs2021_5yr_B19013_14000US36047030600.csv')

In [4]:
income

,geoid,name,B19013001,"B19013001, Error"
0,14000US36005000100,"Census Tract 1, Bronx, NY",NaN,NaN
1,14000US36005000200,"Census Tract 2, Bronx, NY",70867.0,25423.0
2,14000US36005000400,"Census Tract 4, Bronx, NY",98090.0,18180.0
3,14000US36005001600,"Census Tract 16, Bronx, NY",40033.0,9907.0
4,14000US36005001901,"Census Tract 19.01, Bronx, NY",55924.0,12028.0
...,...,...,...,...
2322,14000US36085030302,"Census Tract 303.02, Richmond, NY",85842.0,18154.0
2323,14000US36085031901,"Census Tract 319.01, Richmond, NY",NaN,NaN
2324,14000US36085031902,"Census Tract 319.02, Richmond, NY",76066.0,35257.0
2325,14000US36085032300,"Census Tract 323, Richmond, NY",86471.0,25095.0


In [5]:
# rename the columns 

# 3rd column is Median Household Income in the Past 12 Months (In 2021 Inflation-adjusted Dollars)

income.rename(columns={'B19013001': 'median_household_income'}, inplace=True)

In [6]:
income = income[income['median_household_income'] > 0]

In [7]:
income['geoid'] = income['geoid'].str.replace('14000US', '')

/var/folders/bp/5dzvdsg55xn25gp9vjh1f4xc0000gn/T/ipykernel_28918/2929573091.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income['geoid'] = income['geoid'].str.replace('14000US', '')


In [8]:
income['geoid'] = income['geoid'].astype(int)

/var/folders/bp/5dzvdsg55xn25gp9vjh1f4xc0000gn/T/ipykernel_28918/2520246159.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income['geoid'] = income['geoid'].astype(int)


In [9]:
income['COUNTY'] = income['geoid'].astype(str).str.slice(2,5).astype(int)
income['TRACT'] = income['geoid'].astype(str).str.slice(5).astype(int)
income

/var/folders/bp/5dzvdsg55xn25gp9vjh1f4xc0000gn/T/ipykernel_28918/4201568212.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income['COUNTY'] = income['geoid'].astype(str).str.slice(2,5).astype(int)
/var/folders/bp/5dzvdsg55xn25gp9vjh1f4xc0000gn/T/ipykernel_28918/4201568212.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income['TRACT'] = income['geoid'].astype(str).str.slice(5).astype(int)


,geoid,name,median_household_income,"B19013001, Error",COUNTY,TRACT
1,36005000200,"Census Tract 2, Bronx, NY",70867.0,25423.0,5,200
2,36005000400,"Census Tract 4, Bronx, NY",98090.0,18180.0,5,400
3,36005001600,"Census Tract 16, Bronx, NY",40033.0,9907.0,5,1600
4,36005001901,"Census Tract 19.01, Bronx, NY",55924.0,12028.0,5,1901
5,36005001902,"Census Tract 19.02, Bronx, NY",60804.0,12156.0,5,1902
...,...,...,...,...,...,...
2320,36085029106,"Census Tract 291.06, Richmond, NY",127671.0,25994.0,85,29106
2321,36085030301,"Census Tract 303.01, Richmond, NY",95913.0,6123.0,85,30301
2322,36085030302,"Census Tract 303.02, Richmond, NY",85842.0,18154.0,85,30302
2324,36085031902,"Census Tract 319.02, Richmond, NY",76066.0,35257.0,85,31902


Although all my data is at the tract level, the notation doesn't match. Fixing that below. 

In [10]:
tracts = pd.read_csv('ny_census_tracts.csv')

In [11]:
tracts

,CTLabel,BoroCode,BoroName,CT2020,BoroCT2020,CDEligibil,NTAName,NTA2020,CDTA2020,CDTANAME,GEOID,Shape_Leng,Shape_Area
0,1.00,1,Manhattan,100,1000100,NaN,The Battery-Governors Island-Ellis Island-Libe...,MN0191,MN01,MN01 Financial District-Tribeca (CD 1 Equivalent),36061000100,11023.048501,1.844421e+06
1,2.01,1,Manhattan,201,1000201,NaN,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061000201,4754.495247,9.723121e+05
2,6.00,1,Manhattan,600,1000600,NaN,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061000600,6976.286456,2.582705e+06
3,14.01,1,Manhattan,1401,1001401,NaN,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001401,5075.332000,1.006117e+06
4,14.02,1,Manhattan,1402,1001402,NaN,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001402,4459.156019,1.226206e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2320,77.00,5,Staten Island,7700,5007700,NaN,St. George-New Brighton,SI0101,SI01,SI01 North Shore (CD 1 Equivalent),36085007700,7325.091410,2.674908e+06
2321,19.02,4,Queens,1902,4001902,NaN,Long Island City-Hunters Point,QN0201,QN02,QN02 Long Island City-Sunnyside-Woodside (CD 2...,36081001902,5659.156615,1.909110e+06
2322,171.01,4,Queens,17101,4017101,NaN,Sunnyside Yards (South),QN0261,QN02,QN02 Long Island City-Sunnyside-Woodside (CD 2...,36081017101,22732.905385,8.783519e+06
2323,475.00,4,Queens,47500,4047500,NaN,Elmhurst,QN0401,QN04,QN04 Elmhurst-Corona (CD 4 Approximation),36081047500,8890.142310,3.028836e+06


In [12]:
# only keep the columns we need

tracts = tracts[['CTLabel', 'CT2020', 'GEOID']]

In [13]:
tracts.rename(columns={'GEOID': 'geoid'}, inplace=True)

/var/folders/bp/5dzvdsg55xn25gp9vjh1f4xc0000gn/T/ipykernel_28918/766813932.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracts.rename(columns={'GEOID': 'geoid'}, inplace=True)


In [14]:
# find column types

tracts.dtypes

CTLabel    float64
CT2020       int64
geoid        int64
dtype: object

In [15]:
# join tracts to income based on the geoid using an inner join

income_tracts = pd.merge(income, tracts, on='geoid', how='outer', indicator=True)

In [16]:
income_tracts._merge.value_counts()

# confirming that all the tracts in the income df are accounted for 

_merge
both          2196
right_only     129
left_only        0
Name: count, dtype: int64

In [17]:
# only keep the both rows

income_tracts = income_tracts[income_tracts['_merge'] == 'both']

# now I have a dataset I can use to join to the other datasets

In [18]:
income_tracts.head()

,geoid,name,median_household_income,"B19013001, Error",COUNTY,TRACT,CTLabel,CT2020,_merge
0,36005000200,"Census Tract 2, Bronx, NY",70867.0,25423.0,5.0,200.0,2.00,200,both
1,36005000400,"Census Tract 4, Bronx, NY",98090.0,18180.0,5.0,400.0,4.00,400,both
2,36005001600,"Census Tract 16, Bronx, NY",40033.0,9907.0,5.0,1600.0,16.00,1600,both
3,36005001901,"Census Tract 19.01, Bronx, NY",55924.0,12028.0,5.0,1901.0,19.01,1901,both
4,36005001902,"Census Tract 19.02, Bronx, NY",60804.0,12156.0,5.0,1902.0,19.02,1902,both


Reading the Open Streets data

In [19]:
open_streets = pd.read_csv('mean_coords_table_with_geos.csv')

In [20]:
open_streets['FULL_GEOID'] = open_streets['GEOID']
open_streets

,wkt_geom,MEAN_X,MEAN_Y,object_id,lat,lon,GEOID,STATE,COUNTY,TRACT,BLOCK,FULL_GEOID
0,Point (932357.50813676416873932 129799.6779506...,9.323575e+05,129799.677951,1.0,40.52279,-74.186652,360850198004041,36,85,19800,4041,360850198004041
1,Point (952222.48087393492460251 147937.0785196...,9.522225e+05,147937.078520,384.0,40.57267,-74.115286,360850134001007,36,85,13400,1007,360850134001007
2,Point (957307.78094421629793942 167722.3556344...,9.573078e+05,167722.355634,480.0,40.62699,-74.097060,360850059021000,36,85,5902,1000,360850059021000
3,Point (961974.16859200922772288 165665.9565205...,9.619742e+05,165665.956521,501.0,40.62136,-74.080242,360850029002000,36,85,2900,2000,360850029002000
4,Point (962028.83035567402839661 168307.7564461...,9.620288e+05,168307.756446,6.0,40.62861,-74.080054,360850033002000,36,85,3300,2000,360850033002000
...,...,...,...,...,...,...,...,...,...,...,...,...
537,Point (1056074.98569662659429014 158679.738558...,1.056075e+06,158679.738558,325.0,40.60192,-73.741347,360811010021009,36,81,101002,1009,360811010021009
538,Point (1056284.10559147596359253 158493.067578...,1.056284e+06,158493.067578,326.0,40.60141,-73.740596,360811010021003,36,81,101002,1003,360811010021003
539,Point (1056446.62167064100503922 158342.209940...,1.056447e+06,158342.209940,320.0,40.60099,-73.740012,360811010021000,36,81,101002,1000,360811010021000
540,Point (1056606.48847688734531403 158161.324475...,1.056606e+06,158161.324475,321.0,40.60050,-73.739439,360811010021000,36,81,101002,1000,360811010021000


In [21]:
open_streets['GEOID'] = open_streets.GEOID.astype(str).str.slice(0,-4).astype(int)

In [22]:
open_streets

,wkt_geom,MEAN_X,MEAN_Y,object_id,lat,lon,GEOID,STATE,COUNTY,TRACT,BLOCK,FULL_GEOID
0,Point (932357.50813676416873932 129799.6779506...,9.323575e+05,129799.677951,1.0,40.52279,-74.186652,36085019800,36,85,19800,4041,360850198004041
1,Point (952222.48087393492460251 147937.0785196...,9.522225e+05,147937.078520,384.0,40.57267,-74.115286,36085013400,36,85,13400,1007,360850134001007
2,Point (957307.78094421629793942 167722.3556344...,9.573078e+05,167722.355634,480.0,40.62699,-74.097060,36085005902,36,85,5902,1000,360850059021000
3,Point (961974.16859200922772288 165665.9565205...,9.619742e+05,165665.956521,501.0,40.62136,-74.080242,36085002900,36,85,2900,2000,360850029002000
4,Point (962028.83035567402839661 168307.7564461...,9.620288e+05,168307.756446,6.0,40.62861,-74.080054,36085003300,36,85,3300,2000,360850033002000
...,...,...,...,...,...,...,...,...,...,...,...,...
537,Point (1056074.98569662659429014 158679.738558...,1.056075e+06,158679.738558,325.0,40.60192,-73.741347,36081101002,36,81,101002,1009,360811010021009
538,Point (1056284.10559147596359253 158493.067578...,1.056284e+06,158493.067578,326.0,40.60141,-73.740596,36081101002,36,81,101002,1003,360811010021003
539,Point (1056446.62167064100503922 158342.209940...,1.056447e+06,158342.209940,320.0,40.60099,-73.740012,36081101002,36,81,101002,1000,360811010021000
540,Point (1056606.48847688734531403 158161.324475...,1.056606e+06,158161.324475,321.0,40.60050,-73.739439,36081101002,36,81,101002,1000,360811010021000


In [23]:
# this gives us the number of open streets per tract

open_streets_count = open_streets[['GEOID']].value_counts()

In [24]:
# save as a dataframe

open_streets_count = pd.DataFrame(open_streets_count).reset_index()

In [25]:
open_streets_count

,GEOID,count
0,36081038302,14
1,36081016900,11
2,36061007300,10
3,36081101002,9
4,36081029100,8
...,...,...
228,36047051800,1
229,36047052900,1
230,36047055200,1
231,36047057300,1


In [26]:
# open_streets_count.reset_index(inplace=True)

In [27]:
open_streets_count.rename(columns={'GEOID': 'geoid'}, inplace=True)

In [28]:
open_streets_count

,geoid,count
0,36081038302,14
1,36081016900,11
2,36061007300,10
3,36081101002,9
4,36081029100,8
...,...,...
228,36047051800,1
229,36047052900,1
230,36047055200,1
231,36047057300,1


In [29]:
income_tracts = income_tracts.drop(columns='TRACT')
income_tracts.rename(columns={'CT2020': 'TRACT'}, inplace=True)

In [30]:
# drop the _merge column

income_tracts.drop(columns=['_merge'], inplace=True)

In [31]:
income_tracts.head()

,geoid,name,median_household_income,"B19013001, Error",COUNTY,CTLabel,TRACT
0,36005000200,"Census Tract 2, Bronx, NY",70867.0,25423.0,5.0,2.00,200
1,36005000400,"Census Tract 4, Bronx, NY",98090.0,18180.0,5.0,4.00,400
2,36005001600,"Census Tract 16, Bronx, NY",40033.0,9907.0,5.0,16.00,1600
3,36005001901,"Census Tract 19.01, Bronx, NY",55924.0,12028.0,5.0,19.01,1901
4,36005001902,"Census Tract 19.02, Bronx, NY",60804.0,12156.0,5.0,19.02,1902


In [32]:
open_streets_count

,geoid,count
0,36081038302,14
1,36081016900,11
2,36061007300,10
3,36081101002,9
4,36081029100,8
...,...,...
228,36047051800,1
229,36047052900,1
230,36047055200,1
231,36047057300,1


In [33]:
merged = pd.merge(open_streets_count, income_tracts, on='geoid', how='outer', indicator=True)
merged

,geoid,count,name,median_household_income,"B19013001, Error",COUNTY,CTLabel,TRACT,_merge
0,36081038302,14.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,36081016900,11.0,"Census Tract 169, Queens, NY",77027.0,15493.0,81.0,169.00,16900.0,both
2,36061007300,10.0,"Census Tract 73, New York, NY",142409.0,42874.0,61.0,73.00,7300.0,both
3,36081101002,9.0,"Census Tract 1010.02, Queens, NY",38395.0,32774.0,81.0,1010.02,101002.0,both
4,36081029100,8.0,"Census Tract 291, Queens, NY",58226.0,5107.0,81.0,291.00,29100.0,both
...,...,...,...,...,...,...,...,...,...
2199,36085029106,NaN,"Census Tract 291.06, Richmond, NY",127671.0,25994.0,85.0,291.06,29106.0,right_only
2200,36085030301,NaN,"Census Tract 303.01, Richmond, NY",95913.0,6123.0,85.0,303.01,30301.0,right_only
2201,36085030302,NaN,"Census Tract 303.02, Richmond, NY",85842.0,18154.0,85.0,303.02,30302.0,right_only
2202,36085031902,NaN,"Census Tract 319.02, Richmond, NY",76066.0,35257.0,85.0,319.02,31902.0,right_only


In [34]:
merged['_merge'].value_counts()

_merge
right_only    1971
both           225
left_only        8
Name: count, dtype: int64

In [35]:
merged.query('name.isna()')

,geoid,count,name,median_household_income,"B19013001, Error",COUNTY,CTLabel,TRACT,_merge
0,36081038302,14.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only
34,36081017101,4.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only
84,36081064102,2.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only
120,36085005902,1.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only
129,36081019901,1.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only
131,36081009900,1.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only
153,36061016600,1.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only
224,36047044902,1.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [36]:
# show the value counts of the _merge column

merged._merge.value_counts()

# this means that there are five tracts that have open streets but no median household income data

_merge
right_only    1971
both           225
left_only        8
Name: count, dtype: int64

In [37]:
# keep only the rows where the _merge column is 'both'

# merged = merged[merged['_merge'] == 'both']

In [38]:
merged

,geoid,count,name,median_household_income,"B19013001, Error",COUNTY,CTLabel,TRACT,_merge
0,36081038302,14.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,36081016900,11.0,"Census Tract 169, Queens, NY",77027.0,15493.0,81.0,169.00,16900.0,both
2,36061007300,10.0,"Census Tract 73, New York, NY",142409.0,42874.0,61.0,73.00,7300.0,both
3,36081101002,9.0,"Census Tract 1010.02, Queens, NY",38395.0,32774.0,81.0,1010.02,101002.0,both
4,36081029100,8.0,"Census Tract 291, Queens, NY",58226.0,5107.0,81.0,291.00,29100.0,both
...,...,...,...,...,...,...,...,...,...
2199,36085029106,NaN,"Census Tract 291.06, Richmond, NY",127671.0,25994.0,85.0,291.06,29106.0,right_only
2200,36085030301,NaN,"Census Tract 303.01, Richmond, NY",95913.0,6123.0,85.0,303.01,30301.0,right_only
2201,36085030302,NaN,"Census Tract 303.02, Richmond, NY",85842.0,18154.0,85.0,303.02,30302.0,right_only
2202,36085031902,NaN,"Census Tract 319.02, Richmond, NY",76066.0,35257.0,85.0,319.02,31902.0,right_only


In [39]:
# save the merged dataset

merged.to_csv('final_data.csv', index=False)